In [0]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [0]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [0]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash"

gpt_name = "Maggie"
claude_name = "Eddie"
gemini_name = "Jean"

gpt_system = "You are a chatbot that impersonates the late great actress Maggie Smith \
with her dry sharp British wit. Your name is Maggie, and you are a good friend of Eddie and Jean \
but that doesn't stop you to tease and try to outwit them both. \
Respond in short phrases."

claude_system = "You are a chatbot that impersonates Eddie Murphy \
with his high-energy, fast talking American humor. Your name is Eddie, and you a good friend of Maggie and Jean \
but that doesn't stop you to try to outdo them both. \
Respond in short phrases."

gemini_system = "You are a chatbot that impersonates Jean Dujardin \
with his charming, slapstick, deadpan irony kind of humor. Your name is Jean, and you are a good friend of Maggie and Eddie \
but that doesn't stop you to try to outcharm them both. \
Respond in short phrases."

In [0]:
# Connect to OpenAI, Anthropic and Google

openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=gemini_system
)

In [0]:
def call_gpt():    
    messages = [{"role": "system", "content": gpt_system}]
    for gpt_m, claude_m, gemini_m in zip(gpt_messages, claude_messages,gemini_messages): 
        messages.append({"role": "assistant", "content": gpt_m})
        messages.append({"role": "user", "content": concatenate_user_msg(claude_m,claude_name,gemini_m,gemini_name)})        
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [0]:
def concatenate_user_msg(msg1, name1, msg2, name2):
    return name1 + ' said: ' + msg1 + '. \n\nThen ' + name2 + ' said: ' + msg2 + '.'

In [0]:
def call_claude():
    messages = []
    for gpt_m, claude_m,gemini_m in zip(gpt_messages, claude_messages,gemini_messages):
        messages.append({"role": "user", "content": concatenate_user_msg(gpt_m,gpt_name,gemini_m,gemini_name)})
        messages.append({"role": "assistant", "content": claude_m})        
    messages.append({"role": "user", "content": gemini_messages[-1]})  
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [0]:
def call_gemini():
    messages = []
    for gpt_m, claude_m, gemini_m in zip(gpt_messages, claude_messages,gemini_messages):
        messages.append({"role": "user", "parts": concatenate_user_msg(gpt_m,gpt_name,claude_m,claude_name)})        
        messages.append({"role": "assistant", "parts": [{"text": gemini_m}]})        
    messages.append({"role": "user", "parts": [{"text": gemini_messages[-1]}]})   
    response = gemini.generate_content(messages)
    return response.candidates[0].content.parts[0].text

In [0]:
gpt_messages = ["Well, look what the cat dragged in. And here I thought you'd all been lost at sea."]
claude_messages = ["Awww man, c'mere! I ain't seen y'all in forever — you still look crazy!"]
gemini_messages = ["Mes amis! At last! I thought you had forgotten the most handsome of your friends!"]

In [0]:
print(f"Maggie:\n{gpt_messages[0]}\n")
print(f"Eddie:\n{claude_messages[0]}\n")
print(f"Jean:\n{gemini_messages[0]}\n")
for i in range(5):
    gpt_next = call_gpt()
    print(f"Maggie:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Eddie:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next=call_gemini()
    print(f"Jean:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)